In [ ]:
# 📦 STANDARD LIBRARY IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import yaml
from pathlib import Path
from datetime import datetime

# 🎯 DCS VISUALIZATION IMPORTS
from discernus.visualization.dcs_plots import (
    plot_dcs_framework,
    plot_coordinate_space,
    plot_competitive_dynamics,
    plot_temporal_evolution,
    setup_publication_style,
    extract_framework_anchors,
    calculate_signature_coordinates
)

# 🎨 PUBLICATION-READY VISUALIZATION SETUP
setup_publication_style('nature')  # Academic journal standards
plt.rcParams['figure.figsize'] = (10, 8)  # Larger for DCS plots

print("✅ Standard libraries loaded successfully!")
print("✅ DCS plotting functions loaded!")
print("🎯 Ready for publication-quality DCS analysis")


In [ ]:
# 🔍 EXPERIMENT DATA LOADING
# This section automatically finds and loads experiment data

# Find experiment data files in current directory
current_dir = Path('.')
run_metadata_file = current_dir / 'run_metadata.json'

# Load run metadata if available
if run_metadata_file.exists():
    with open(run_metadata_file, 'r') as f:
        run_metadata = json.load(f)
    print(f"✅ Run metadata loaded: {run_metadata.get('job_id', 'Unknown')}")
    
    # Find experiment YAML file
    experiment_path = run_metadata.get('experiment_path')
    if experiment_path and Path(experiment_path).exists():
        with open(experiment_path, 'r') as f:
            experiment_def = yaml.safe_load(f)
        print(f"✅ Experiment definition loaded: {experiment_def.get('name', 'Unknown')}")
    else:
        print("⚠️ Experiment YAML not found - using metadata only")
        experiment_def = {}
else:
    print("⚠️ No run metadata found")
    print("💡 Tip: This template works best with experiment results")
    run_metadata = {}
    experiment_def = {}

# Display basic experiment info
print("\n📊 Experiment Overview:")
print(f"   Name: {experiment_def.get('name', 'Unknown')}")
print(f"   Framework: {experiment_def.get('framework', {}).get('name', 'Unknown')}")
print(f"   Models: {run_metadata.get('models_used', ['Not specified'])}")
print(f"   Job ID: {run_metadata.get('job_id', 'Unknown')}")


In [ ]:
# 📈 RESULTS DATA LOADING
# Load experiment results from database or files

# Try to load from run metadata first
experiment_results = run_metadata.get('results', {})
condition_results = experiment_results.get('condition_results', [])

if condition_results:
    print(f"✅ Found {len(condition_results)} model conditions")
    
    # Convert to DataFrame for easy analysis
    model_data = []
    for condition in condition_results:
        model_name = condition.get('condition_identifier', 'Unknown')
        centroid = condition.get('centroid', [0, 0])
        total_analyses = condition.get('total_analyses', 0)
        
        model_data.append({
            'model': model_name,
            'centroid_x': centroid[0] if len(centroid) > 0 else 0,
            'centroid_y': centroid[1] if len(centroid) > 1 else 0,
            'total_analyses': total_analyses,
            'magnitude': np.sqrt(centroid[0]**2 + centroid[1]**2) if len(centroid) >= 2 else 0
        })
    
    # Create DataFrame
    results_df = pd.DataFrame(model_data)
    print("\n📊 Model Results Summary:")
    display(results_df)
    
else:
    print("⚠️ No condition results found")
    print("💡 Creating sample data for template demonstration")
    
    # Create sample data so template can still be used
    results_df = pd.DataFrame({
        'model': ['Sample Model A', 'Sample Model B', 'Sample Model C'],
        'centroid_x': [0.5, -0.3, 0.1],
        'centroid_y': [0.2, 0.7, -0.4],
        'total_analyses': [10, 15, 8],
        'magnitude': [0.54, 0.76, 0.41]
    })
    
    print("\n📊 Sample Data for Template:")
    display(results_df)


In [ ]:
# 🎯 DCS FRAMEWORK VISUALIZATION
# Publication-quality DCS analysis using extracted framework and results

# Extract framework configuration
framework_config = experiment_def.get('framework', {})
framework_name = framework_config.get('name', 'Unknown Framework')

print(f"📊 Creating DCS visualization for: {framework_name}")

# Check if we have proper framework configuration
if framework_config and ('anchors' in framework_config or 'axes' in framework_config):
    try:
        # Create main DCS framework visualization
        fig, ax = plot_dcs_framework(
            framework_config=framework_config,
            experiment_results=run_metadata.get('results', {}),
            title=f"DCS Analysis: {framework_name}",
            figsize=(12, 10)
        )
        
        plt.show()
        print("✅ DCS Framework visualization complete!")
        
        # If framework has competitive relationships, create competitive dynamics plot
        if framework_config.get('competitive_relationships'):
            print("\n🔄 Creating competitive dynamics analysis...")
            
            # Extract signatures from results
            signatures = {}
            for _, row in results_df.iterrows():
                model_name = row['model']
                signatures[model_name] = np.array([row['centroid_x'], row['centroid_y']])
            
            fig_comp, (ax1, ax2) = plot_competitive_dynamics(
                framework_config=framework_config,
                signatures=signatures,
                figsize=(14, 6)
            )
            
            plt.show()
            print("✅ Competitive dynamics visualization complete!")
        
    except Exception as e:
        print(f"⚠️ Error creating DCS visualization: {e}")
        print("📋 Falling back to basic coordinate plot...")
        
        # Fallback to basic plot
        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        
        # Plot model centroids with enhanced styling
        colors = plt.cm.tab10(np.linspace(0, 1, len(results_df)))
        for idx, (_, row) in enumerate(results_df.iterrows()):
            ax.scatter(row['centroid_x'], row['centroid_y'], 
                      s=200, color=colors[idx], alpha=0.8, 
                      label=row['model'], edgecolors='black', linewidth=1)
            
            ax.annotate(row['model'], 
                       (row['centroid_x'], row['centroid_y']),
                       xytext=(5, 5), textcoords='offset points',
                       fontsize=10, ha='left', weight='bold')

        # Add coordinate system
        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
        
        ax.set_xlabel('DCS Dimension 1', fontsize=12)
        ax.set_ylabel('DCS Dimension 2', fontsize=12)
        ax.set_title(f'Model Comparison: {framework_name}', fontsize=14, pad=20)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
        
        plt.tight_layout()
        plt.show()
        print("✅ Basic coordinate visualization complete")

else:
    print("⚠️ No framework configuration found - using basic scatter plot")
    
    # Basic scatter plot for when no framework is available
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(results_df)))
    for idx, (_, row) in enumerate(results_df.iterrows()):
        ax.scatter(row['centroid_x'], row['centroid_y'], 
                  s=200, color=colors[idx], alpha=0.8, 
                  label=row['model'], edgecolors='black', linewidth=1)
        
        ax.annotate(row['model'], 
                   (row['centroid_x'], row['centroid_y']),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=10, ha='left')

    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    ax.set_xlabel('X Coordinate', fontsize=12)
    ax.set_ylabel('Y Coordinate', fontsize=12)
    ax.set_title('Model Comparison: Coordinate Space', fontsize=14, pad=20)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    print("✅ Basic visualization complete")

print("\n💡 DCS Visualization Features:")
print("   • Automatic framework detection and visualization")
print("   • Publication-ready styling (Nature journal standards)")
print("   • Competitive dynamics analysis (if configured)")
print("   • Fallback to basic plots if needed")
print("   • Fully customizable using standard matplotlib")


In [ ]:
# 📈 TEMPORAL EVOLUTION ANALYSIS
# Advanced temporal analysis if temporal data is available

print("🔄 Checking for temporal analysis capabilities...")

# Check if we have temporal data in results
temporal_data_available = False
temporal_df = None

# Look for temporal indicators in the data
if 'temporal_phase' in results_df.columns or 'date' in results_df.columns or 'time' in results_df.columns:
    temporal_data_available = True
    temporal_df = results_df.copy()
    print("✅ Temporal data detected!")
    
    # Try to create temporal evolution visualization
    if framework_config and ('anchors' in framework_config or 'axes' in framework_config):
        try:
            print("📊 Creating temporal evolution analysis...")
            
            # Determine temporal column
            if 'temporal_phase' in temporal_df.columns:
                phase_col = 'temporal_phase'
            elif 'date' in temporal_df.columns:
                phase_col = 'date'
                # Create phases from dates if needed
                if temporal_df[phase_col].dtype == 'object':
                    # Try to parse dates and create phases
                    try:
                        temporal_df['date'] = pd.to_datetime(temporal_df['date'])
                        temporal_df['temporal_phase'] = pd.cut(temporal_df['date'], 
                                                             bins=3, 
                                                             labels=['early', 'middle', 'late'])
                        phase_col = 'temporal_phase'
                    except:
                        phase_col = 'date'
            else:
                phase_col = 'time'
            
            # Add signature coordinates if not present
            if 'signature_coords' not in temporal_df.columns:
                # Create from centroid data
                temporal_df['signature_coords'] = temporal_df.apply(
                    lambda row: [row['centroid_x'], row['centroid_y']], axis=1
                )
            
            # Create temporal evolution plot
            fig_temporal, phase_data = plot_temporal_evolution(
                framework_config=framework_config,
                temporal_data=temporal_df,
                phase_column=phase_col,
                signature_column='signature_coords',
                figsize=(16, 10)
            )
            
            plt.show()
            print("✅ Temporal evolution analysis complete!")
            
            # Print phase analysis summary
            if phase_data:
                print(f"\n📊 Temporal Phase Analysis:")
                for phase, data in phase_data.items():
                    centroid = data['centroid']
                    intensity = np.linalg.norm(centroid)
                    print(f"   • {phase}: {data['count']} points, "
                          f"centroid ({centroid[0]:.3f}, {centroid[1]:.3f}), "
                          f"intensity {intensity:.3f}")
            
        except Exception as e:
            print(f"⚠️ Error creating temporal evolution plot: {e}")
            print("💡 Temporal data found but visualization failed - check data format")
            
            # Show what temporal data we have
            print(f"\n📋 Available temporal columns: {[col for col in temporal_df.columns if 'temp' in col.lower() or 'date' in col.lower() or 'time' in col.lower()]}")
            if phase_col in temporal_df.columns:
                print(f"📋 {phase_col} values: {temporal_df[phase_col].unique()}")
    else:
        print("⚠️ Temporal data found but no framework configuration available")
        print("💡 Basic temporal analysis could be implemented here")
        
        # Simple temporal plot
        if 'date' in temporal_df.columns:
            fig, ax = plt.subplots(1, 1, figsize=(12, 6))
            
            # Plot centroids over time
            ax.scatter(range(len(temporal_df)), temporal_df['centroid_x'], 
                      label='X Coordinate', alpha=0.7)
            ax.scatter(range(len(temporal_df)), temporal_df['centroid_y'], 
                      label='Y Coordinate', alpha=0.7)
            
            ax.set_xlabel('Time Points', fontsize=11)
            ax.set_ylabel('Coordinate Value', fontsize=11)
            ax.set_title('Coordinate Evolution Over Time', fontsize=12, weight='bold')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            print("✅ Basic temporal plot complete")

else:
    print("ℹ️ No temporal data detected in results")
    print("💡 For temporal analysis, ensure your data includes:")
    print("   • 'temporal_phase' column with phase labels")
    print("   • 'date' column with datetime information")
    print("   • 'time' column with time indicators")

print(f"\n🎯 Temporal Analysis Status: {'Available' if temporal_data_available else 'Not Available'}")
if temporal_data_available:
    print("💡 Advanced temporal evolution shows discourse dynamics over time")
    print("📊 Includes phase centroids, trajectory analysis, and intensity changes")
else:
    print("💡 Add temporal columns to your data for advanced temporal analysis")


In [ ]:
# 📁 EXPORT RESULTS
# Save results for publication or further analysis

# Create exports directory
export_dir = Path('exports')
export_dir.mkdir(exist_ok=True)

# Export DataFrame to CSV
results_csv = export_dir / 'model_results.csv'
results_df.to_csv(results_csv, index=False)
print(f"✅ Results exported to: {results_csv}")

# Export visualization
fig_path = export_dir / 'coordinate_visualization.png'
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for idx, (_, row) in enumerate(results_df.iterrows()):
    ax.scatter(row['centroid_x'], row['centroid_y'], 
              s=200, color=colors[idx], alpha=0.7, 
              label=row['model'], edgecolors='black', linewidth=1)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Model Comparison: {framework_name}', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close()
print(f"✅ Visualization exported to: {fig_path}")

# Export summary report
report_path = export_dir / 'analysis_summary.txt'
with open(report_path, 'w') as f:
    f.write(f"Experiment Analysis Summary\n")
    f.write(f"=========================\n\n")
    f.write(f"Experiment: {experiment_def.get('name', 'Unknown')}\n")
    f.write(f"Framework: {framework_name}\n")
    f.write(f"Job ID: {run_metadata.get('job_id', 'Unknown')}\n")
    f.write(f"Analysis Date: {datetime.now().isoformat()}\n\n")
    f.write(f"Models Analyzed: {len(results_df)}\n")
    f.write(f"Total Analyses: {results_df['total_analyses'].sum()}\n")
    f.write(f"Mean Magnitude: {results_df['magnitude'].mean():.3f}\n")
    f.write(f"\nModel Details:\n")
    for _, row in results_df.iterrows():
        f.write(f"  {row['model']}: ({row['centroid_x']:.3f}, {row['centroid_y']:.3f})\n")

print(f"✅ Summary report exported to: {report_path}")
print("\n🎉 All exports complete!")
print("💡 Ready for academic publication or further analysis")
